<a href="https://colab.research.google.com/github/yxyfer/EPTIA_NLP3/blob/main/lab02/SCIA_NLP3_Lab02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os 
import pandas as pd 

from IPython.display import HTML

In [2]:
%%capture
! pip install fasttext

# I. Language detection (24 points)

## Setup 

In [3]:
%%capture
! git clone https://github.com/MastafaF/LanguageDetection.git

In [3]:
os.listdir("./LanguageDetection")

['data_train.txt',
 'dataset.csv',
 'all.txt',
 'langdetect.bin',
 'sentences.csv',
 '.gitignore',
 'valid.txt',
 '.git',
 'train.txt',
 'README.md',
 'sentences.tar.bz2',
 'data_test_no_label.txt',
 'sentences.tar',
 'LICENSE',
 'dataset.csv.zip',
 'data_test.txt']

In [4]:
# CD the LanguageDetection folder - we are working in the below folder now
os.chdir("./LanguageDetection")

In [6]:
# only if not installed
! unzip dataset.csv.zip

Archive:  dataset.csv.zip
replace dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


## Data Exploration Analysis

In [5]:
data = pd.read_csv("./dataset.csv")

In [6]:
# Sample of the data
HTML(data[data.language == "Chinese"].sample().to_html())

,Text,language
8349,张伯声（年月日－年月日），中国构造地质学家。生于河南荥阳。曾就读于河南留学欧美预备学校，年毕业于清华学校。年获美国芝加哥大学化学系学士学位。年当选为中国科学院学部委员院士。曾任西北大学教授、副校长，西安地质学院教授、院长、名誉院长。,Chinese


### Question 1.1. Describe the distribution of languages and give at least two comments about the dataset. (1 point)

In [7]:
################################################
# your implementation goes here

def get_language_distrib(data):
    print(data.groupby('language').size())

get_language_distrib(data)
##################################################

language
Arabic        1000
Chinese       1000
Dutch         1000
English       1000
Estonian      1000
French        1000
Hindi         1000
Indonesian    1000
Japanese      1000
Korean        1000
Latin         1000
Persian       1000
Portugese     1000
Pushto        1000
Romanian      1000
Russian       1000
Spanish       1000
Swedish       1000
Tamil         1000
Thai          1000
Turkish       1000
Urdu          1000
dtype: int64


### Question 1.2. Do the appropriate pre-processing to maximise the accuracy of language detection. What is your strategy? (1 point)

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split

In [9]:
X=data['Text']
y=data['language']

In [10]:
X[0]

'klement gottwaldi surnukeha palsameeriti ning paigutati mausoleumi surnukeha oli aga liiga hilja ja oskamatult palsameeritud ning hakkas ilmutama lagunemise tundemärke  aastal viidi ta surnukeha mausoleumist ära ja kremeeriti zlíni linn kandis aastatel – nime gottwaldov ukrainas harkivi oblastis kandis zmiivi linn aastatel – nime gotvald'

In [11]:
import gc

#### Question 1.2.1. Preprocess X

In [12]:
import re
from gensim.parsing import (
    preprocess_string,
    strip_numeric,
    strip_multiple_whitespaces,
    #strip_punctuation,
  )
from sklearn.feature_extraction.text import CountVectorizer


################################################
# your implementation goes here

def pre_process(text):
  CUSTOM_FILTERS = [
    lambda s: s.lower(),
    lambda s: re.sub(r'[!@#$(),"%^*?:;~`]', ' ', s),
    strip_numeric,
    #strip_punctuation, 
    strip_multiple_whitespaces,
  ]
  
  return ' '.join(
      preprocess_string(text, CUSTOM_FILTERS)
    )

X = X.apply(pre_process)

X_copy = X

# cv = CountVectorizer()
cv = CountVectorizer(min_df=5)
cv.fit(X)
X = cv.transform(X)
X = X.toarray()

print(X)
################################################

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


#### Question 1.2.1. Preprocess labels (y)


In [13]:
from sklearn import preprocessing

################################################
# your implementation goes here

le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

print(y)

################################################

[ 4 17 19 ... 16  1 14]


### Question 1.5. Train a model of your choice and describe the accuracy across languages. Use an 80%, 20% train-test split. Performance is not key but explain thoroughly the process and the metric(s) you are tracking. (4 points)

#### 1.5.1. Create & Train Model

In [14]:
%%time
from sklearn.naive_bayes import MultinomialNB

################################################
# your implementation goes here

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

def train_model(X_train, y_train):
  clf = MultinomialNB()
  clf.fit(X_train, y_train)

  return clf

clf = train_model(X_train, y_train)
print(f'General Model Score: {clf.score(X_test, y_test)}')

################################################

General Model Score: 0.9315909090909091
CPU times: user 1min 19s, sys: 1.68 s, total: 1min 21s
Wall time: 1min 21s


#### 1.5.2 Accuracy per Language

In [15]:
################################################
# your implementation goes here

def score_per_lang(y_true, y_hat):
  Dic = [i for i in range(22)]
  language_pos = dict.fromkeys(Dic, 0)
  language_tot = dict.fromkeys(Dic, 0)

  for t, h in zip(y_true, y_hat):
    if h == t:
        language_pos[t] += 1
    language_tot[t] += 1

  accuracies = dict()
  print('Accuracy for each language:')
  for i in range(22):
    lang = le.inverse_transform([i])[0]
    if language_tot[i] != 0:
      accuracy = language_pos[i] / language_tot[i]
    else:
      accuracy = 0
    accuracies[lang] = accuracy
    print(f'{lang:10}: {accuracy:f}')

  return accuracies


y_hat = clf.predict(X_test)
lang_accuracies = score_per_lang(y_hat, y_test)

################################################

Accuracy for each language:
Arabic    : 1.000000
Chinese   : 0.917647
Dutch     : 0.989796
English   : 0.723636
Estonian  : 0.975369
French    : 0.489130
Hindi     : 0.995169
Indonesian: 0.995215
Japanese  : 1.000000
Korean    : 1.000000
Latin     : 0.983146
Persian   : 1.000000
Portugese : 0.989071
Pushto    : 0.994975
Romanian  : 0.994792
Russian   : 0.995074
Spanish   : 0.965000
Swedish   : 0.974747
Tamil     : 1.000000
Thai      : 1.000000
Turkish   : 0.995146
Urdu      : 1.000000


## FastText for language detection

### FastText training setup 

In [18]:
! wget http://downloads.tatoeba.org/exports/sentences.tar.bz2

URL transformed to HTTPS due to an HSTS policy
--2023-01-06 12:58:59--  https://downloads.tatoeba.org/exports/sentences.tar.bz2
Resolving downloads.tatoeba.org (downloads.tatoeba.org)... 94.130.77.194
Connecting to downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172433658 (164M) [application/octet-stream]
Saving to: ‘sentences.tar.bz2’

sentences.tar.bz2   100%[===================>] 164.45M  16.4MB/s    in 11s     

2023-01-06 12:59:11 (15.0 MB/s) - ‘sentences.tar.bz2’ saved [172433658/172433658]



In [19]:
! bunzip2 sentences.tar.bz2

bunzip2: Output file sentences.tar already exists.


In [20]:
! ls

all.txt		 langdetect.bin  sentences.csv	    train.txt
dataset.csv	 LICENSE	 sentences.tar	    valid.txt
dataset.csv.zip  README.md	 sentences.tar.bz2


In [21]:
! tar xvf sentences.tar

sentences.csv


In [22]:
! ls

all.txt		 langdetect.bin  sentences.csv	    train.txt
dataset.csv	 LICENSE	 sentences.tar	    valid.txt
dataset.csv.zip  README.md	 sentences.tar.bz2


In [23]:
! head -10 sentences.csv

1	cmn	我們試試看！
2	cmn	我该去睡觉了。
3	cmn	你在干什麼啊？
4	cmn	這是什麼啊？
5	cmn	今天是６月１８号，也是Muiriel的生日！
6	cmn	生日快乐，Muiriel！
7	cmn	Muiriel现在20岁了。
8	cmn	密码是"Muiriel"。
9	cmn	我很快就會回來。
10	cmn	我不知道。


In [24]:
! awk -F"\t" '{print"__label__"$2" "$3}' < sentences.csv | shuf > all.txt

In [25]:
! ls

all.txt		 langdetect.bin  sentences.csv	    train.txt
dataset.csv	 LICENSE	 sentences.tar	    valid.txt
dataset.csv.zip  README.md	 sentences.tar.bz2


In [26]:
! head -5 all.txt

__label__fra La pièce est très petite, à tel point qu'il n'est pas possible d'ajouter des meubles.
__label__spa Los gemelos se ocupaban del bebé.
__label__por Ele é ucraniano.
__label__ina Il ha multe blau edificios gigantesc in le grande citate.
__label__deu Sie ist vorigen Montag mit ihm einkaufen gegangen.


In [27]:
! head -n 10000 all.txt > valid.txt

In [28]:
! tail -n +10001 all.txt > train.txt

In [29]:
! head -5 train.txt

__label__jpn その日から、不安がつのっていくばかりだった。
__label__por Quanto menor o medo, menor o perigo.
__label__tur Neden Tom bir şey yapmıyor?
__label__eng Tom has never got the credit for anything.
__label__spa John es un chico americano.


### Question 1.6. Train a fasttext model on Tatoeba parallel corpus and check that performance is good. (3 points)

In [16]:
del X
gc.collect()

0

In [17]:
import fasttext

#### Question 1.6.1. Train fasttext model on Tatoeba (2 points)

In [33]:
%%time
import fasttext

# Check the fasttext library and implement the training
###########################################

# your implementation goes here
pass

fmodel = fasttext.train_supervised('train.txt') #, lr=1, epoch=1)

################################################

# @TODO: Save your model when trained 
fmodel.save_model("langdetect.bin")

CPU times: user 1h 40min 15s, sys: 9.02 s, total: 1h 40min 24s
Wall time: 1h 40min 40s


In [27]:
#fmodel = fasttext.load_model('langdetect.bin')
fmodel = fasttext.load_model('/content/drive/MyDrive/ColabNotebooks/NLP3/lab02/langdetect.bin')

In [28]:
# Sanity check 
fmodel.predict("I am French and I love English")

(('__label__eng',), array([1.00000882]))

In [29]:
fmodel.predict("Bonjour je suis")

(('__label__fra',), array([0.99995446]))

#### Question 1.6.2. Evaluate performance of fasttext model on valid.txt (1 point)

In [21]:
%%capture
pip install iso639-lang

In [22]:
from iso639 import Lang

Lang('ara')

Lang(name='Arabic', pt1='ar', pt2b='ara', pt2t='ara', pt3='ara', pt5='')

In [305]:
from iso639 import Lang

class ModelPredict(object):
  def __init__(self, model, le, nb_languages=22):
    self.model = model
    self.le = le
    self.nb_languages = nb_languages

    self.languages = self.get_languages()
    self.flabels = self.format_model_labels()
    self.lang_to_code, self.code_to_lang = self.get_code_to_lang()


  def predict(self, sentence):
    lang_code = self.model.predict(sentence)[0][0].replace('__label__', '')
    return self.code_to_lang.get(lang_code, 'Not Found')

  def predict_set(self, test_set):
    y_hat = []
    for sentence in test_set:
      label = self.predict(sentence)
      if label == 'Not Found':
        y_hat_curr = -1;
      else:
        y_hat_curr = self.le.transform([label])[0]
      y_hat.append(y_hat_curr)

    return y_hat

  ### HELPERS

  def get_languages(self):
    languages = []
    for i in range(self.nb_languages):
      lang = self.le.inverse_transform([i])[0]
      languages.append(lang)

    return languages

  def remove_label(self, str):
    return str.replace('__label__', '')

  def format_model_labels(self):
    labels_raw = sorted(self.model.get_labels())
    labels = list(map(lambda str: self.remove_label(str), labels_raw))
    return labels[1:]

  def get_code_to_lang(self):
    lang_to_code = dict.fromkeys(self.languages, '')

    for lang in self.languages:
      lang_code = Lang('Portuguese').pt3 if lang == 'Portugese' else 'pes' if lang == 'Persian' else (Lang(lang).pt3 if lang != 'Chinese' else 'cmn')
      if lang_code in self.flabels:
        lang_to_code[lang] = lang_code

    code_to_lang = {v: k for k, v in lang_to_code.items()}

    for lang in ['lzh', 'hak', 'cjy', 'cpi', 'hsn', 'gan', 'izh']:
      code_to_lang[lang] = 'Chinese'

    return lang_to_code, code_to_lang
    

In [306]:
mp = ModelPredict(fmodel, le)

In [281]:
print(mp.languages[:5])
print(mp.flabels[:10])

['Arabic', 'Chinese', 'Dutch', 'English', 'Estonian']
['abk', 'acm', 'ady', 'afb', 'afh', 'afr', 'aii', 'ain', 'ajp', 'akl']


In [283]:
mp.predict('Bonjour Comment allez vous')

'French'

In [284]:
# Hint: Create dataframe from valid.txt and evaluate performance 

###########################################

# your implementation goes here

result = fmodel.test('valid.txt')
print("Validation set accuracy:", result[1])

################################################

Validation set accuracy: 0.9904


### Question 1.7. Test your fasttext model on the same dataset as in question 1-5. Compare with your custom model (make sure you use the exact same data for testing). How can you explain the difference in performance between the two models? (3 points)

In [294]:
import fasttext

###########################################

# your implementation goes here

def save_data(X, y, train=False, label=True):
  filename = 'data_train.txt' if train else ('data_test.txt' if label else 'data_test_no_label.txt')
  
  file = open(filename, 'w')
  for x_curr, y_curr in zip(X, y):
    if label:
      lang = le.inverse_transform([y_curr])[0]
      lang_code = Lang('Portuguese').pt3 if lang == 'Portugese' else Lang(lang).pt3
      y_curr = '__label__' + lang_code
      final_string = y_curr + ' ' + x_curr + '\n'
    else:
      final_string = x_curr + '\n'
    
    file.write(final_string)
  file.close()


fX_train, fX_test, fy_train, fy_test = train_test_split(X_copy, y, test_size=0.20)


save_data(fX_train, fy_train, train=True)             # Train with labels
save_data(fX_test, fy_test, train=False)              # Test  with labels
save_data(fX_test, fy_test, train=False, label=False) # Test  no   labels

################################################

In [295]:
result = fmodel.test('data_test.txt')
print("Validation set accuracy:", result[1])

Validation set accuracy: 0.8312593330014932


In [311]:
fy_hat = mp.predict_set(fX_test)
print(-1 in fy_hat)

True


### Question 1.8. Compute your performance metrics yourself and compare with sklearn. (1 point)

#### 1.8.1. Personal Metrics

In [308]:
def accuracy_score_recoded(y_true, y_hat):
  return np.mean(y_true == y_hat)


fast_accuracy = accuracy_score_recoded(fy_test, fy_hat)

print(f'Accuracy Score: {fast_accuracy}\n')

Accuracy Score: 0.8040909090909091



In [309]:
_ = score_per_lang(fy_test, fy_hat)

Accuracy for each language:
Arabic    : 0.985000
Chinese   : 0.005435
Dutch     : 0.960591
English   : 0.995169
Estonian  : 0.830688
French    : 0.990654
Hindi     : 0.980488
Indonesian: 0.903226
Japanese  : 0.821596
Korean    : 0.942708
Latin     : 0.919192
Persian   : 0.994949
Portugese : 0.945545
Pushto    : 0.000000
Romanian  : 0.979275
Russian   : 0.995074
Spanish   : 0.977778
Swedish   : 1.000000
Tamil     : 0.465608
Thai      : 0.100000
Turkish   : 0.979167
Urdu      : 0.883495


#### 1.8.2. sklearn Metrics

In [310]:
from sklearn.metrics import accuracy_score

accuracy_score(fy_test, fy_hat)

0.8040909090909091

# II. Rotate two semantic spaces (23 points) – Not guided coding
